In [ ]:
library(tidymodels)
library(tidyverse)

In [ ]:
library(dplyr)
library(ggplot2)

In [ ]:
install.packages('stacks')
install.packages('rpart')
install.packages('ranger')
install.packages('kknn')

In [ ]:
install.packages('vip')


In [ ]:
install.packages('varImp')

In [ ]:
library(varImp)

In [ ]:
library(vip)

In [ ]:
library(rsample)
library(parsnip)
library(tune)
library(stacks)
library(yardstick)
library(recipes)



In [ ]:
library(themis)

In [ ]:

combined_features_final <- read_csv('../input/preprocessedcombinedfeatures/CombinedFeaturesFinal.csv')

In [ ]:
combined_features_final

In [ ]:
is.na(combined_features_final)

In [ ]:
glimpse(combined_features_final)

In [ ]:
relevant_features <- combined_features_final %>% select(-X1, -gameId, -playId) %>% 
                    mutate(playType = factor(playType), offenseFormation = factor(offenseFormation), personnelO = factor(personnelO), personnelD = factor(personnelD),typeDropback = factor(typeDropback), passResult = factor(passResult))

relevant_features

In [ ]:
#predict playType

relevant_features_split <- relevant_features %>% initial_split(prop=0.75, strata=playType)

relevant_features_training <- training(relevant_features_split)

relevant_features_test <- testing(relevant_features_split)

relevant_features_bootstraps <- bootstraps(relevant_features_training, strata=playType)

relevant_features_folds <- vfold_cv(relevant_features_training, v=10, strata=playType)

relevant_features_validation <- validation_split(relevant_features_training, strata=playType)



relevant_features_recipe <- recipe(playType ~ ., data=relevant_features_training) %>%
                            step_upsample(playType) %>% 
                            step_YeoJohnson(all_numeric(), -all_outcomes()) %>% 
                            step_normalize(all_numeric(), -all_outcomes()) %>% 
                            step_dummy(all_nominal(), -all_outcomes()) %>%
                            step_pca(all_numeric(), -all_outcomes())
                           
                            
                            
                            
                            


glimpse(relevant_features_recipe %>% prep() %>% bake(new_data=relevant_features_training))

relevant_features_recipe %>% prep() %>% bake(new_data=relevant_features_training)


In [ ]:
#RandomForest

rand_forest_model <- rand_forest() %>% set_engine('ranger') %>% set_mode('classification')


rand_forest_workflow <- workflow() %>% add_recipe(relevant_features_recipe) %>% add_model(rand_forest_model)






#stacks

#K-Nearest Neighbor

knn_model <- nearest_neighbor() %>% set_engine('kknn') %>% set_mode('classification')

knn_workflow <- workflow() %>% add_recipe(relevant_features_recipe) %>% add_model(knn_model)




#Decision Tree

decision_tree_model <- decision_tree() %>% set_engine('rpart') %>% set_mode('classification')

decision_tree_workflow <- workflow() %>% add_recipe(relevant_features_recipe) %>% add_model(decision_tree_model)



#mlp Neural Net

mlp_model <- mlp() %>% set_engine('keras') %>% set_mode('classification')

mlp_workflow <- workflow() %>% add_recipe(relevant_features_recipe) %>% add_model(mlp_model)




In [ ]:
rand_forest_workflow



In [ ]:
knn_workflow

decision_tree_workflow

In [ ]:


mlp_workflow

In [ ]:
rand_forest_fit <- rand_forest_workflow %>% parsnip::fit(relevant_features_training)

In [ ]:
mlp_fit <- mlp_workflow %>% parsnip::fit(relevant_features_training)

In [ ]:
rand_forest_fit

In [ ]:
mlp_fit

In [ ]:
random_forest_results <- rand_forest_fit %>% predict(relevant_features_test) %>% bind_cols(relevant_features_test) %>% rename(random_forest_predictions = .pred_class) %>% select(playType, random_forest_predictions)

random_forest_metrics <- random_forest_results %>% metrics(playType, random_forest_predictions)

random_forest_metrics

random_forest_results

In [ ]:
control_grid <- control_stack_grid()

In [ ]:
decision_tree_resampled <- fit_resamples(decision_tree_workflow, relevant_features_bootstraps, control = control_grid)






In [ ]:
knn_resampled <- fit_resamples(knn_workflow, relevant_features_bootstraps, control = control_grid)


In [ ]:
stacked_classifier <- stacks() %>% add_candidates(decision_tree_resampled) %>% add_candidates(knn_resampled)

In [ ]:
stacked_classifier_fit <- stacked_classifier %>% blend_predictions() %>% fit_members()

In [ ]:
stacked_classifier_fit

In [ ]:
stacked_results <- stacked_classifier_fit %>% predict(relevant_features_test) %>% bind_cols(relevant_features_test) %>% rename(stacked_ensemble_predictions = .pred_class) %>% select(playType, stacked_ensemble_predictions)

stacked_metrics <- stacked_results %>% metrics(playType, stacked_ensemble_predictions)


stacked_metrics

stacked_results




In [ ]:
mlp_results <- mlp_fit %>% predict(relevant_features_test) %>% bind_cols(relevant_features_test) %>% rename(mlp_predictions = .pred_class) %>% select(playType, mlp_predictions)

mlp_metrics <- mlp_results %>% metrics(playType, mlp_predictions)


mlp_metrics

mlp_results




In [ ]:
relevant_features

In [ ]:
#predict typeDropback

dropback_features_split <- relevant_features %>% initial_split(prop=0.75, strata = typeDropback)

dropback_features_train <- training(dropback_features_split)

dropback_features_test <- testing(dropback_features_split)

dropback_features_bootstraps <- bootstraps(dropback_features_train, strata=typeDropback)



In [ ]:
#predict typeDropback

dropback_features_recipe <- recipe(typeDropback ~ ., data=dropback_features_train) %>%
                            step_upsample(typeDropback) %>% 
                            step_YeoJohnson(all_numeric(), -all_outcomes()) %>% 
                            step_normalize(all_numeric(), -all_outcomes()) %>% 
                            step_dummy(all_nominal(), -all_outcomes()) %>%
                            step_pca(all_numeric(), -all_outcomes())


glimpse(dropback_features_recipe %>% prep() %>% bake(new_data=dropback_features_train))

dropback_features_recipe %>% prep() %>% bake(new_data=dropback_features_train)

In [ ]:

#RandomForest




rand_forest_dropback_workflow <- workflow() %>% add_recipe(dropback_features_recipe) %>% add_model(rand_forest_model)






#stacks

#K-Nearest Neighbor


knn_dropback_workflow <- workflow() %>% add_recipe(dropback_features_recipe) %>% add_model(knn_model)




#Decision Tree



decision_tree_dropback_workflow <- workflow() %>% add_recipe(dropback_features_recipe) %>% add_model(decision_tree_model)



#mlp Neural Net



mlp_dropback_workflow <- workflow() %>% add_recipe(dropback_features_recipe) %>% add_model(mlp_model)

print("Done!")

In [ ]:

#fit randomForest

rand_forest_dropback_fit <- rand_forest_dropback_workflow %>% parsnip::fit(dropback_features_train)

rand_forest_dropback_fit

In [ ]:
#fit mlp

mlp_dropback_fit <- mlp_dropback_workflow %>% parsnip::fit(dropback_features_train)

mlp_dropback_fit

In [ ]:
rand_forest_dropback_results <- rand_forest_dropback_fit %>% predict(dropback_features_test) %>% bind_cols(dropback_features_test) %>% rename(rand_forest_predictions = .pred_class) %>% select(typeDropback, rand_forest_predictions)

rand_forest_dropback_metrics <- rand_forest_dropback_results %>% metrics(typeDropback, rand_forest_predictions)

rand_forest_dropback_metrics

rand_forest_dropback_results

In [ ]:
mlp_dropback_results <- mlp_dropback_fit %>% predict(dropback_features_test) %>% bind_cols(dropback_features_test) %>% rename(mlp_predictions = .pred_class) %>% select(typeDropback, mlp_predictions)

mlp_dropback_metrics <- mlp_dropback_results %>% metrics(typeDropback, mlp_predictions)


mlp_dropback_metrics #MLPs are NOT the move on tabular data

mlp_dropback_results

In [ ]:
dropback_control_grid <- control_stack_grid()

In [ ]:
decision_tree_dropback_fit <- fit_resamples(decision_tree_dropback_workflow, dropback_features_bootstraps, control=dropback_control_grid)

In [ ]:
knn_dropback_fit <- fit_resamples(knn_dropback_workflow, dropback_features_bootstraps, control=dropback_control_grid)

In [ ]:
dropback_stack <- stacks() %>% add_candidates(decision_tree_dropback_fit) %>% add_candidates(knn_dropback_fit)



In [ ]:
dropback_stack_fit <- dropback_stack %>% blend_predictions() %>% fit_members()

In [ ]:
dropback_stack_fit

In [ ]:
dropback_stack_results <- dropback_stack_fit %>% predict(dropback_features_test) %>% bind_cols(dropback_features_test) %>% rename(stacked_dropback_preds = .pred_class) %>% select(typeDropback, stacked_dropback_preds)

dropback_stack_metrics <- dropback_stack_results %>% metrics(typeDropback, stacked_dropback_preds)

dropback_stack_metrics

dropback_stack_results

In [ ]:

personnel_split <- relevant_features %>% initial_split(prop=3/4, strata=personnelO)

personnel_train <- training(personnel_split)

personnel_test <- testing(personnel_split)

personnel_bootstraps <- bootstraps(personnel_train, strata=personnelO)






                       

In [ ]:
personnel_recipe <- recipe(personnelO ~ ., data=personnel_train) %>%
                        step_YeoJohnson(all_numeric(), -all_outcomes()) %>%
                        step_normalize(all_numeric(), -all_outcomes()) %>%
                        step_medianimpute(all_numeric(), -all_outcomes()) %>%
                        step_pca(all_numeric(), -all_outcomes()) 
                        
                        



In [ ]:

glimpse(personnel_recipe %>% prep() %>% bake(new_data=personnel_train))

In [ ]:

rand_forest_personnel_workflow <- workflow() %>% add_recipe(personnel_recipe) %>% add_model(rand_forest_model)



In [ ]:
rand_forest_personnel_fit <- rand_forest_personnel_workflow %>% parsnip::fit(personnel_train)

In [ ]:
rand_forest_personnel_results <- rand_forest_personnel_fit %>% predict(personnel_test) %>% bind_cols(personnel_test) %>% rename(rand_forest_personnel_preds = .pred_class) %>% select(personnelO, rand_forest_personnel_preds)

rand_forest_personnel_metrics <- rand_forest_personnel_results %>% metrics(personnelO, rand_forest_personnel_preds)





In [ ]:

rand_forest_personnel_metrics

In [ ]:
rand_forest_personnel_results